In [1]:
import requests
from lxml import etree
import time

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36",
    "Referer": "https://www.w3resource.com/java-exercises/index.php",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Cookie": "cookieconsent_dismissed=yes; _ga=GA1.1.951155699.1662966123; __gads=ID=e4a9ceed58af0db0-22e8c0e363d600fa:T=1662966123:RT=1662966123:S=ALNI_Mb8ekgDHdeUAOiYpiP2b5aOv2bZoQ; __gpi=UID=000009bc08a12cd1:T=1662966123:RT=1665412442:S=ALNI_MZsq5XVOMV4gXv29odQuujZZsSr2g; _ga_HCW88J419P=GS1.1.1665459271.4.0.1665459271.0.0.0"
}

In [3]:
index_response = requests.get("https://www.w3resource.com/java-exercises/index.php")

In [4]:
index_html = etree.HTML(index_response.content.decode("utf-8"))

In [5]:
exercises_category = index_html.xpath("//ul[@class=\"nav nav-list\"]/li/a/text()")[2:17]
exercises_category_href = index_html.xpath("//ul[@class=\"nav nav-list\"]/li/a/@href")[2:17]

In [6]:
exercise_links = dict(zip(map(lambda category: category.strip() ,exercises_category), 
         map(lambda position: "https://www.w3resource.com" + position, exercises_category_href)))

In [7]:
exercise_links

{'Basic Part-I': 'https://www.w3resource.com/java-exercises/basic/index.php',
 'Basic Part-II': 'https://www.w3resource.com/java-exercises/basic/index1.php',
 'Data Types': 'https://www.w3resource.com/java-exercises/datatypes/index.php',
 'Conditional Statement': 'https://www.w3resource.com/java-exercises/conditional-statement/index.php',
 'Array': 'https://www.w3resource.com/java-exercises/array/index.php',
 'String': 'https://www.w3resource.com/java-exercises/string/index.php',
 'Date Time': 'https://www.w3resource.com/java-exercises/datetime/index.php',
 'Method': 'https://www.w3resource.com/java-exercises/method/index.php',
 'Numbers': 'https://www.w3resource.com/java-exercises/numbers/index.php',
 'File Input-Output': 'https://www.w3resource.com/java-exercises/io/index.php',
 'Collection': 'https://www.w3resource.com/java-exercises/collection/index.php',
 'Math': 'https://www.w3resource.com/java-exercises/math/index.php',
 'Sorting': 'https://www.w3resource.com/java-exercises/sort

In [8]:
exercise_solution_links_prefix = {}
for key in exercise_links.keys():
    link = exercise_links[key]
    exercise_solution_links_prefix[key] = link[0: link.rfind('/')]
exercise_solution_links_prefix

{'Basic Part-I': 'https://www.w3resource.com/java-exercises/basic',
 'Basic Part-II': 'https://www.w3resource.com/java-exercises/basic',
 'Data Types': 'https://www.w3resource.com/java-exercises/datatypes',
 'Conditional Statement': 'https://www.w3resource.com/java-exercises/conditional-statement',
 'Array': 'https://www.w3resource.com/java-exercises/array',
 'String': 'https://www.w3resource.com/java-exercises/string',
 'Date Time': 'https://www.w3resource.com/java-exercises/datetime',
 'Method': 'https://www.w3resource.com/java-exercises/method',
 'Numbers': 'https://www.w3resource.com/java-exercises/numbers',
 'File Input-Output': 'https://www.w3resource.com/java-exercises/io',
 'Collection': 'https://www.w3resource.com/java-exercises/collection',
 'Math': 'https://www.w3resource.com/java-exercises/math',
 'Sorting': 'https://www.w3resource.com/java-exercises/sorting',
 'Search': 'https://www.w3resource.com/java-exercises/search',
 'Regular Expression': 'https://www.w3resource.com/j

In [9]:
strong_element_texts = ["Input", "Output"]
all_exercises = {
    "Collection": {}
}
for category in exercise_links.keys():
    link = exercise_links[category]
    exercise_page_response = requests.get(link, headers=headers)
    exercise_page_html = etree.HTML(exercise_page_response.content.decode("utf-8"))
    if category == "Collection":
        article_elements = exercise_page_html.xpath("//article/*")[0:-4]
        subhead = None
        for element in article_elements:
            if element.tag == "h2" and element.attrib.get("class") == "subheading":
                subhead = etree.tostring(element, encoding="utf-8", method="text").decode("utf-8")
                all_exercises["Collection"][subhead] = []
            elif subhead != None:
                exercise = {}
                exercise["qid"] = int(element.find("./strong").text.strip().strip("."))
                exercise["content_element"] = element
                all_exercises["Collection"][subhead].append(exercise)
    else:
        start = 5 if len(exercise_page_html.xpath("//article/p[@class=\"heading\"]")) == 0 else 6
        if category == "String":
            start = 6
        article_elements = exercise_page_html.xpath("//article/*")[start:-4]
        exercises = []
        last_exercise = {}
        for element in article_elements:
            strong_element = element.find("./strong")
            if strong_element != None:
                strong_element_text = strong_element.text.strip().strip(":")
            if strong_element != None and strong_element_text not in strong_element_texts:
                exercise = {}
                exercise["qid"] = int(element.find("./strong").text.strip().strip("."))
                exercise["content_elements"] = [element]
                last_exercise = exercise
                exercises.append(exercise)
            else:
                last_exercise["content_elements"].append(element)
        all_exercises[category] = exercises
    time.sleep(2)

In [10]:
exercise_number = 0
for key in all_exercises.keys():
    if key == "Collection":
        for subhead in all_exercises[key].keys():
            exercise_number += len(all_exercises[key][subhead])
    else:
        exercise_number += len(all_exercises[key])
print("Number of exercises: {}".format(exercise_number))

Number of exercises: 813


In [11]:
# 可能含有的元素：Input Data、Expected Output、Test Data、

In [13]:
for category in all_exercises.keys():
    solution_prefix = exercise_solution_links_prefix[category]
    if category != "Collection":
        category_exercises = all_exercises[category]
        for exercise in category_exercises:
            exercise_elements = exercise["content_elements"]
            for element in exercise_elements:
                a_tags = element.findall("./a")
                for a_tag in a_tags:
                    a_text = etree.tostring(a_tag, encoding="utf-8", method="text").decode("utf-8")
                    if a_text.find("Click me to see the solution") != -1:
                        exercise["solution_link"] = solution_prefix + "/" + a_tag.attrib.get("href")
    else:
        collection_exercises = all_exercises["Collection"]
        for collection_sub_exercises in collection_exercises.values():
            for exercise in collection_sub_exercises:
                exercise_element = exercise["content_element"]
                a_tags = exercise_element.findall("./a")
                for a_tag in a_tags:
                    a_text = etree.tostring(a_tag, encoding="utf-8", method="text").decode("utf-8")
                    if a_text.find("Click me to see the solution") != -1:
                        exercise["solution_link"] = solution_prefix + "/" + a_tag.attrib.get("href")
        

In [15]:
for category in all_exercises.keys():
    if category != "Collection":
        category_exercises = all_exercises[category]
        for exercise in category_exercises:
            text = ""
            for element in exercise["content_elements"]:
                text_part = etree.tostring(element, encoding="utf-8", method="text").decode("utf-8")
                text_part = text_part.replace("Go to the editor", "").replace("Click me to see the solution", "").strip()
                if text_part:
                    text += text_part
            exercise["content"] = text
    else:
        collection_exercises = all_exercises["Collection"]
        for collection_sub_exercises in collection_exercises.values():
            for exercise in collection_sub_exercises:
                text = etree.tostring(exercise["content_element"], encoding="utf-8", method="text").decode("utf-8")
                text = text.replace("Go to the editor", "").replace("Click me to see the solution", "").strip()
                exercise["content"] = text

In [17]:
import random

for category in all_exercises.keys():
    if category != "Collection":
        category_exercises = all_exercises[category]
        for exercise in category_exercises:
            solution_link = exercise.get("solution_link", None)
            if solution_link:
                solution_page = requests.get(solution_link, headers=headers)
                solution_html = etree.HTML(solution_page.content.decode("utf-8"))
                solution_code_ele = solution_html.xpath("//pre[@class]/code")[0]
                solution_code = etree.tostring(solution_code_ele, encoding="utf-8", method="text").decode("utf-8")
                exercise["solution_code"] = solution_code
            else:
                exercise["solution_code"] = ""
                exercise["error"] = "no solution code"
            time.sleep(random.randint(1, 5))
    else:
        collection_exercises = all_exercises["Collection"]
        for collection_sub_exercises in collection_exercises.values():
            for exercise in collection_sub_exercises:
                solution_link = exercise.get("solution_link", None)
                if solution_link:
                    solution_page = requests.get(solution_link, headers=headers)
                    solution_html = etree.HTML(solution_page.content.decode("utf-8"))
                    solution_code_ele = solution_html.xpath("//pre[@class]/code")[0]
                    solution_code = etree.tostring(solution_code_ele, encoding="utf-8", method="text").decode("utf-8")
                    exercise["solution_code"] = solution_code
                else:
                    exercise["solution_code"] = ""
                    exercise["error"] = "no solution code"
                time.sleep(random.randint(1, 5))


In [19]:
for category in all_exercises.keys():
    if category != "Collection":
        category_exercises = all_exercises[category]
        for exercise in category_exercises:
            solution_code = exercise.get("error", None)
            if solution_code:
                print("{}: {}".format(category, exercise["qid"]))
    else:
        collection_exercises = all_exercises["Collection"]
        for collection_sub_exercises in collection_exercises.values():
            for exercise in collection_sub_exercises:
                solution_code = exercise.get("error", None)
                if solution_code:
                    print("{}: {}".format(category, exercise["qid"]))

Basic Part-I: 125
Basic Part-I: 126
Basic Part-I: 127


In [36]:
all_exercises["Basic Part-I"][124]["solution_link"] = \
        "https://www.w3resource.com/java-exercises/basic/java-basic-exercise-125.php"
all_exercises["Basic Part-I"][125]["solution_link"] = \
        "https://www.w3resource.com/java-exercises/basic/java-basic-exercise-126.php"
all_exercises["Basic Part-I"][126]["solution_link"] = \
        "https://www.w3resource.com/java-exercises/basic/java-basic-exercise-127.php"

In [38]:
solution_page = requests.get(all_exercises["Basic Part-I"][124]["solution_link"], headers=headers)
solution_html = etree.HTML(solution_page.content.decode("utf-8"))
solution_code_ele = solution_html.xpath("//pre[@class]/code")[0]
solution_code = etree.tostring(solution_code_ele, encoding="utf-8", method="text").decode("utf-8")
all_exercises["Basic Part-I"][124]["solution_code"] = solution_code

solution_page = requests.get(all_exercises["Basic Part-I"][125]["solution_link"], headers=headers)
solution_html = etree.HTML(solution_page.content.decode("utf-8"))
solution_code_ele = solution_html.xpath("//pre[@class]/code")[0]
solution_code = etree.tostring(solution_code_ele, encoding="utf-8", method="text").decode("utf-8")
all_exercises["Basic Part-I"][125]["solution_code"] = solution_code

solution_page = requests.get(all_exercises["Basic Part-I"][126]["solution_link"], headers=headers)
solution_html = etree.HTML(solution_page.content.decode("utf-8"))
solution_code_ele = solution_html.xpath("//pre[@class]/code")[0]
solution_code = etree.tostring(solution_code_ele, encoding="utf-8", method="text").decode("utf-8")
all_exercises["Basic Part-I"][126]["solution_code"] = solution_code

In [39]:
data = {"Collection": {}}
for category in all_exercises.keys():
    if category != "Collection":
        category_exercises = all_exercises[category]
        data[category] = list(map(lambda exercise: dict(exercise_number=exercise["qid"], 
            exercise_content=exercise["content"], exercise_solution_code=exercise["solution_code"],
            exercise_solution_link=exercise["solution_link"]), category_exercises))
    else:
        collection_exercises = all_exercises["Collection"]
        for collection_subhead in collection_exercises.keys():
            collection_sub_exercises = all_exercises["Collection"][collection_subhead]
            data["Collection"][collection_subhead] = list(map(lambda exercise: dict(exercise_number=exercise["qid"], 
                exercise_content=exercise["content"], exercise_solution_code=exercise["solution_code"],
                exercise_solution_link=exercise["solution_link"]), collection_sub_exercises))

In [41]:
import os
import json

data_path = os.path.join(os.path.abspath(""), "data.json")
f = open(data_path, "w")
f.write(json.dumps(data))
f.close() 

In [42]:
with open(data_path, "r") as f:
    data2 = json.load(f)

In [46]:
data2["Regular Expression"][29]

{'exercise_number': 30,
 'exercise_content': '30. Write a Java program to insert a dash (-) between an upper case letter and a lower case letter in a given string.',
 'exercise_solution_code': 'public class test { \r\n   public static void main(String[] args) {\t   \r\n\t    String text = "Python Exercises";\r\n\t\tSystem.out.println("Original string: "+text);\r\n\t\tSystem.out.println("Insert a dash between an upper case letter and a lower case letter in the said string:\\n"+validate(text));\r\n\t\ttext = "The quick brown Fox jumps over the lazy Dog.";\r\n\t\tSystem.out.println("\\nOriginal string: "+text);\r\n\t\tSystem.out.println("Insert a dash between an upper case letter and a lower case letter in the said string:\\n"+validate(text));\t\r\n\t\ttext = "java exercises";\r\n\t\tSystem.out.println("\\nOriginal string: "+text);\r\n\t\tSystem.out.println("Insert a dash between an upper case letter and a lower case letter in the said string:\\n"+validate(text));\r\n        }\r\n\r\n   p

In [49]:
data2["Regular Expression"][29]["exercise_solution_link"] = \
    "https://www.w3resource.com/java-exercises/re/java-re-exercise-30.php"

In [50]:
data2["Regular Expression"][29]

{'exercise_number': 30,
 'exercise_content': '30. Write a Java program to insert a dash (-) between an upper case letter and a lower case letter in a given string.',
 'exercise_solution_code': 'public class test { \r\n   public static void main(String[] args) {\t   \r\n\t    String text = "Python Exercises";\r\n\t\tSystem.out.println("Original string: "+text);\r\n\t\tSystem.out.println("Insert a dash between an upper case letter and a lower case letter in the said string:\\n"+validate(text));\r\n\t\ttext = "The quick brown Fox jumps over the lazy Dog.";\r\n\t\tSystem.out.println("\\nOriginal string: "+text);\r\n\t\tSystem.out.println("Insert a dash between an upper case letter and a lower case letter in the said string:\\n"+validate(text));\t\r\n\t\ttext = "java exercises";\r\n\t\tSystem.out.println("\\nOriginal string: "+text);\r\n\t\tSystem.out.println("Insert a dash between an upper case letter and a lower case letter in the said string:\\n"+validate(text));\r\n        }\r\n\r\n   p

In [58]:
data2["Basic Part-II"][29]

{'exercise_number': 180,
 'exercise_content': '180. Write a Java program to swap every two adjacent nodes of a given linked list.Expected Output:Original Linked list:\r\n10->20->30->40->50\r\n\r\nAfter swiping Linked list becomes:\r\n20->10->40->30->50',
 'exercise_solution_code': 'import java.util.*;\r\npublic class Solution {  \r\n public static void main(String[] args) {\r\n        String str1 = "The length of last word";\r\n\t\tSystem.out.println("Original String: "+str1);\r\n        System.out.println("Length of the last word of the above string: "+length_Of_last_word(str1));\r\n    }\r\n\r\n    public static int length_Of_last_word(String str1) {\r\n        int length_word = 0;\r\n        String[] words = str1.split(" ");\r\n        if(words.length>0) {\r\n            length_word = words[words.length-1].length();\t\t\t\r\n        } else {\r\n            length_word = 0;\r\n        }\r\n        return length_word;\r\n    }\r\n}\r\n\r\n',
 'exercise_solution_link': 'https://www.w3r

In [59]:
f = open(data_path, "w")
f.write(json.dumps(data2))
f.close() 